In [1]:
import findspark
findspark.init("/Users/mihae/Downloads/spark-3.1.2-bin-hadoop3.2")
findspark.find

<function findspark.find()>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark

In [3]:
spark = SparkSession.builder \
.master("local") \
.appName("Sparky") \
.config("spark.executor.memory", "1gb") \
.getOrCreate()

sc = spark.sparkContext

In [4]:
df = spark.read.format("CSV").option("header", "true").load("titanic.csv")

df = df.withColumn("Survived", df["Survived"].cast(IntegerType()))
df = df.withColumn("Pclass", df["Pclass"].cast(IntegerType()))
df = df.withColumn("Name", df["Name"].cast(StringType()))
df = df.withColumn("Sex", df["Sex"].cast(StringType()))
df = df.withColumn("Age", df["Age"].cast(IntegerType()))
df = df.withColumn("Siblings/Spouses Aboard", df["Siblings/Spouses Aboard"].cast(IntegerType()))
df = df.withColumn("Parents/Children Aboard", df["Parents/Children Aboard"].cast(IntegerType()))
df = df.withColumn("Fare", df["Fare"].cast(FloatType()))

df.show()

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|
|       1|     1|Mrs. Jacques Heat...|female| 35|                      1|                      0|   53.1|
|       0|     3|Mr. William Henry...|  male| 35|                      0|                      0|   8.05|
|       0|     3|     Mr. James Moran|  male| 27|                      0|                      0| 8.4583|
|       0|     1|Mr. Timothy J McC...|  male| 

In [5]:
#3A

df_survival_rate_class_sex = df.select("Survived", "Pclass", "Sex")
df_survival_rate_class_sex = df_survival_rate_class_sex.groupBy("Pclass", "Sex").avg("Survived").toDF("Pclass", "Sex", "Survival Percent")
df_survival_rate_class_sex.show()

+------+------+-------------------+
|Pclass|   Sex|   Survival Percent|
+------+------+-------------------+
|     2|female| 0.9210526315789473|
|     3|  male|0.13702623906705538|
|     1|  male|0.36885245901639346|
|     3|female|                0.5|
|     1|female| 0.9680851063829787|
|     2|  male| 0.1574074074074074|
+------+------+-------------------+



In [6]:
#3B

df_children = df.select("Survived", "Pclass", "Age")
df_children = df_children.filter((df.Pclass == 3) & (df.Age <= 10))

df_children.show()

children_count = df_children.count()
survived = df_children.filter(df_children.Survived == 1).count()
rate = survived/children_count*100
rate=str(rate)
print (str("the survival rate is: " + rate + "%"))

+--------+------+---+
|Survived|Pclass|Age|
+--------+------+---+
|       0|     3|  2|
|       1|     3|  4|
|       0|     3|  2|
|       0|     3|  8|
|       0|     3|  7|
|       0|     3|  4|
|       1|     3|  7|
|       0|     3|  2|
|       1|     3|  2|
|       0|     3|  9|
|       0|     3|  5|
|       0|     3|  1|
|       1|     3|  9|
|       0|     3|  4|
|       1|     3|  1|
|       0|     3|  5|
|       0|     3|  8|
|       0|     3|  9|
|       1|     3|  4|
|       0|     3|  2|
+--------+------+---+
only showing top 20 rows

the survival rate is: 41.509433962264154%


In [7]:
#3C

df_expected_fare = df.select("Pclass", "Fare")
df_expected_fare = df_expected_fare.groupBy("Pclass").avg("Fare").toDF("class", "Average fare expected")
df_expected_fare.show()

+-----+---------------------+
|class|Average fare expected|
+-----+---------------------+
|    1|    84.15468752825701|
|    3|   13.707707501045244|
|    2|    20.66218318109927|
+-----+---------------------+



In [8]:
spark.stop()